# Bader charge analysis

## Visualizing the WorkGraph Builder



In [1]:
from workgraph_collections.ase.espresso.bader import bader_workgraph
from aiida import load_profile
load_profile()

node = bader_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.bader import bader_workgraph

wg = bader_workgraph()
wg.to_html()

/home/xing/miniconda3/envs/aiida/lib/python3.11/site-packages/aiida/engine/processes/ports.py:148: UserWarning: default of input port `parent_folder_name` is a `Node` instance, which can lead to unexpected side effects. It is advised to use a lambda instead, e.g.: `default=lambda: orm.Int(5)`.
  warnings.warn(UserWarning(message))


## Example: Silicon

### Prepare the inputs and submit the workflow


In [5]:
from ase.build import molecule
from aiida import load_profile

from workgraph_collections.ase.espresso.bader import bader_workgraph

load_profile()

atoms = molecule('H2O', vacuum=3.0)
atoms.pbc = True


metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"H": "H.pbe-rrkjus_psl.1.0.0.UPF", 
                    "O": "O.pbe-n-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
scf_input_data = {
    "control": {"calculation": "scf",
                },
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold"
                },
}

inputs = {
    "scf": {
        "input_data": scf_input_data,
        "kpts": (1, 1, 1),
        "computer": "localhost",
        "metadata": metadata
    },
    "pp_valence": {
        "input_data": {
                        "INPUTPP": {"plot_num": 0},
                        "PLOT": {"iflag": 3,
                                 "output_format": 6,
                                 "fileout": "charge_density.cube",
                                 },
                    },
        "computer": "localhost",
        "metadata": metadata
    },
    "pp_all": {
        "input_data": {
                        "INPUTPP": {"plot_num": 21},
                        "PLOT": {"iflag": 3,
                                 "output_format": 6,
                                 "fileout": "charge_density.cube",
                                 },
                    },
        "computer": "localhost",
        "metadata": metadata
    },
    "bader": {
        "computer": "localhost",
        "metadata": metadata
    }
}
#------------------------- Set the inputs -------------------------
wg = bader_workgraph(atoms=atoms,
                    pw_command="mpirun -np 1 pw.x",
                    pp_command="mpirun -np 1 pp.x",
                    bader_command="bader",
                    pseudopotentials=pseudopotentials,
                    pseudo_dir=pseudo_dir,
                    inputs=inputs)
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=200)
#------------------------- Print the output -------------------------
print('bader: \n {}'.format(wg.nodes['bader'].outputs["result"].value.value))



WorkGraph node created, PK: 53009
bader: 
 [7.997613, 0.001169, 0.001216]
